In [1]:
#%% Setting Up
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import networkx as nx
import rioxarray as rxr

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely.geometry import Polygon

import glob
import os
import itertools
import tqdm
import gc
import time
import pickle

from joblib import Parallel, delayed

import rioxarray as rxr

import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/datadir/assets/defaults.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

print("Setting up...")

Setting up...


In [4]:
#%% Region-Specific: CAMELS-US
DIRNAME = '03min_GloFAS_CAMELS-US'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': -130,
    'miny': 20,
    'maxx': -65,
    'maxy': 50
}

camels_graph = pd.read_csv(os.path.join(SAVE_PATH, 'nested_gauges', 'graph_attributes_with_nesting.csv'), index_col=0)
camels_graph.index = camels_graph.index.map(lambda x: str(x).zfill(8))
camels_graph['huc_02'] = camels_graph['huc_02'].map(lambda x: str(x).zfill(2))
# camels_graph = camels_graph[camels_graph['nesting'].isin(['not_nested', 'nested_downstream'])]
camels_graph = camels_graph.reset_index()
print(f"Number of catmt's with nesting: {len(camels_graph)}")

Number of catmt's with nesting: 395


In [5]:
def idx_to_map(ds, var_name):
    lats = ds.lat.values
    lons = ds.lon.values
    catmt_var_map = xr.DataArray(
        np.zeros((len(lats), len(lons)), dtype = np.float32)*np.nan,
        dims = ['lat', 'lon'],
        coords = {'lat': lats, 'lon': lons}
    )
    for idx in ds.idx.values:
        lat, lon = ds['idx2lat'].sel(idx = idx).values, ds['idx2lon'].sel(idx = idx).values
        catmt_var_map.loc[dict(lat = lat, lon = lon)] = ds[var_name].sel(idx = idx).values
    return catmt_var_map

START_DATE = pd.Timestamp('1998-01-01')
END_DATE = pd.Timestamp('2022-12-31')

# Helper Functions

In [6]:
def get_fao_pet(
    surface_pressure_mean: pd.Series,
    temperature_2m_mean: pd.Series,
    dewpoint_temperature_2m_mean: pd.Series,
    u_component_of_wind_10m_mean: pd.Series,
    v_component_of_wind_10m_mean: pd.Series,
    surface_net_solar_radiation_mean: pd.Series,
    surface_net_thermal_radiation_mean: pd.Series,
) -> pd.Series:
    # windspeed2m_m_s
    temp_windspeed10m_m_s = np.sqrt(u_component_of_wind_10m_mean**2 + v_component_of_wind_10m_mean**2)
    windspeed2m_m_s = temp_windspeed10m_m_s * 4.87 / (np.log(67.8 * 10 - 5.42))

    # net_radiation_MJ_m2
    net_radiation_mj_m2 = ((surface_net_solar_radiation_mean + surface_net_thermal_radiation_mean) * 24 / 1e6)

    # Convert units
    surface_pressure_kpa = surface_pressure_mean / 1e3  # convert Pa to kPa
    temperature2m_c = temperature_2m_mean - 273.15  # convert K to C
    dewpoint2m_c = dewpoint_temperature_2m_mean - 273.15  # convert K to C

    # Constants.
    lmbda = 2.45  # Latent heat of vaporization [MJ kg -1] (simplification in the FAO PenMon (latent heat of about 20°C)
    cp = 1.013e-3  # Specific heat at constant pressure [MJ kg-1 °C-1]
    eps = 0.622  # Ratio molecular weight of water vapour/dry air

    # Soil heat flux density [MJ m-2 day-1] - set to 0 following eq 42 in FAO
    soil_heat_flux = np.zeros_like(surface_pressure_kpa)

    # Atmospheric pressure [kPa] eq 7 in FAO.
    P_kpa = surface_pressure_kpa

    # Psychrometric constant (gamma symbol in FAO) eq 8 in FAO.
    psychometric_kpa_c = cp * P_kpa / (eps * lmbda)

    # Saturation vapour pressure, eq 11 in FAO.
    svp_kpa = 0.6108 * np.exp((17.27 * temperature2m_c) / (temperature2m_c + 237.3))

    # Delta (slope of saturation vapour pressure curve) eq 13 in FAO.
    delta_kpa_c = 4098.0 * svp_kpa / (temperature2m_c + 237.3)**2

    # Actual vapour pressure, eq 14 in FAO.
    avp_kpa = 0.6108 * np.exp((17.27 * dewpoint2m_c) / (dewpoint2m_c + 237.3))

    # Saturation vapour pressure deficit.
    svpdeficit_kpa = svp_kpa - avp_kpa

    # Calculate ET0, equation 6 in FAO
    numerator = (0.408 * delta_kpa_c * (net_radiation_mj_m2 - soil_heat_flux) + psychometric_kpa_c *
                 (900 / (temperature2m_c + 273)) * windspeed2m_m_s * svpdeficit_kpa)
    denominator = delta_kpa_c + psychometric_kpa_c * (1 + 0.34 * windspeed2m_m_s)

    ET0_mm_day = numerator / denominator

    return ET0_mm_day

In [7]:
from typing import Dict, List
from numba import njit

@njit
def _split_list(a_list: List) -> List:
    """Splits list into list of lists, where each list contains subsequent numbers."""
    new_list = []
    start = 0
    for index, value in enumerate(a_list):
        if index < len(a_list) - 1:
            if a_list[index + 1] > value + 1:
                end = index + 1
                new_list.append(a_list[start:end])
                start = end
        else:
            new_list.append(a_list[start:len(a_list)])
    return new_list

def _get_moisture_and_seasonality_index(precipitation, pet) -> tuple[float, float]:

    mean_monthly_precip = precipitation.groupby(precipitation.index.month).mean()
    mean_monthly_pet = pet.groupby(pet.index.month).mean()

    # Average annual moisture index (see Knoben)
    p_gt_et = 1 - mean_monthly_pet.loc[mean_monthly_precip > mean_monthly_pet] / mean_monthly_precip.loc[
        mean_monthly_precip > mean_monthly_pet]
    srs = pd.Series(np.zeros((12), dtype=np.float32), index=mean_monthly_pet.index, name='dummy')
    p_eq_et = srs.loc[mean_monthly_precip == mean_monthly_pet]
    p_lt_et = mean_monthly_precip.loc[mean_monthly_precip < mean_monthly_pet] / mean_monthly_pet.loc[
        mean_monthly_precip < mean_monthly_pet] - 1
    monthly_moisture_index = pd.concat([p_gt_et, p_eq_et, p_lt_et])

    annual_moisture_index = monthly_moisture_index.mean()

    # Seasonality (see Knoben)
    seasonality = monthly_moisture_index.max() - monthly_moisture_index.min()

    return annual_moisture_index, seasonality

def get_climate_indices(df: pd.DataFrame) -> Dict[str, float]:
    # Convert units
    df['temperature_2m_mean'] = df['temperature_2m_mean'] - 273.15  # Convert K to C

    # Mean daily precip
    p_mean = df["total_precipitation_sum"].mean()
    # Mean daily PET
    pet_mean_era5 = df["potential_evaporation_sum_ERA5_LAND"].mean()
    pet_mean_fao = df["potential_evaporation_sum_FAO_PENMAN_MONTEITH"].mean()

    # Aridity index
    aridity_era5 = pet_mean_era5 / p_mean
    aridity_fao = pet_mean_fao / p_mean

    # Compute moistuer and seasonality index once with ERA5 PET and once with FAO PM PET
    annual_moisture_index_era5, seasonality_era5 = _get_moisture_and_seasonality_index(
        precipitation=df["total_precipitation_sum"], pet=df["potential_evaporation_sum_ERA5_LAND"])
    annual_moisture_index_fao, seasonality_fao = _get_moisture_and_seasonality_index(
        precipitation=df["total_precipitation_sum"], pet=df["potential_evaporation_sum_FAO_PENMAN_MONTEITH"])
    
    # Fraction of mean monthly precipipitation falling as snow (see Knoben)
    mean_monthly_precip = df["total_precipitation_sum"].groupby(df.index.month).mean()
    mean_monthly_temp = df["temperature_2m_mean"].groupby(df.index.month).mean()
    frac_snow = mean_monthly_precip.loc[mean_monthly_temp < 0].sum() / mean_monthly_precip.sum()

    high_prec_freq = len(df.loc[df["total_precipitation_sum"] >= 5 * p_mean]) / len(df)
    low_prec_freq = len(df.loc[df["total_precipitation_sum"] < 1]) / len(df)

    precip = df["total_precipitation_sum"].values
    idx = np.where(precip < 1)[0]
    groups = _split_list(idx)
    if groups:
        low_precip_dur = np.mean(np.array([len(p) for p in groups]))
    else:
        low_precip_dur = 0.0

    idx = np.where(precip >= 5 * p_mean)[0]
    groups = _split_list(idx)
    if groups:
        high_prec_dur = np.mean(np.array([len(p) for p in groups]))
    else:
        high_prec_dur = 0.0

    # climate_indices = {
    #     'p_mean': p_mean,
    #     'pet_mean_ERA5_LAND': pet_mean_era5,
    #     'pet_mean_FAO_PM': pet_mean_fao,
    #     'aridity_ERA5_LAND': aridity_era5,
    #     'aridity_FAO_PM': aridity_fao,
    #     'frac_snow': frac_snow,
    #     'moisture_index_ERA5_LAND': annual_moisture_index_era5,
    #     'seasonality_ERA5_LAND': seasonality_era5,
    #     'moisture_index_FAO_PM': annual_moisture_index_fao,
    #     'seasonality_FAO_PM': seasonality_fao,
    #     'high_prec_freq': high_prec_freq,
    #     'high_prec_dur': high_prec_dur,
    #     'low_prec_freq': low_prec_freq,
    #     'low_prec_dur': low_precip_dur
    # }
    climate_indices_dict = {
        'p_mean': p_mean,
        'pet_mean_ERA5': pet_mean_era5,
        'pet_mean_FAO_PM': pet_mean_fao,
        'aridity_ERA5': aridity_era5,
        'aridity_FAO_PM': aridity_fao,
        'frac_snow': frac_snow,
        'moisture_index_ERA5': annual_moisture_index_era5,
        'seasonality_ERA5': seasonality_era5,
        'moisture_index_FAO_PM': annual_moisture_index_fao,
        'seasonality_FAO_PM': seasonality_fao,
        'high_prec_freq': high_prec_freq,
        'high_prec_dur': high_prec_dur,
        'low_prec_freq': low_prec_freq,
        'low_prec_dur': low_precip_dur
    }

    return climate_indices_dict

# Execution

In [63]:
def process_catmt(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    catmt = catmt[sorted(catmt.data_vars)]

    START_DATE = pd.Timestamp('1999-01-01')
    END_DATE = pd.Timestamp('2009-12-31')
    catmt_slice = catmt.sel(time=slice(START_DATE, END_DATE)).copy()

    ERA5_var_names = [
        'dynamic_ERA5_total_precipitation',
        'dynamic_ERA5_potential_evaporation',
        'dynamic_ERA5_2m_temperature',
        'dynamic_ERA5_surface_pressure',
        'dynamic_ERA5_2m_dewpoint_temperature',
        'dynamic_ERA5_10m_u_component_of_wind',
        'dynamic_ERA5_10m_v_component_of_wind',
        'dynamic_ERA5_surface_net_solar_radiation',
        'dynamic_ERA5_surface_net_thermal_radiation'
    ]

    catmt_slice = catmt_slice[ERA5_var_names]
    for varname in ['dynamic_ERA5_surface_net_solar_radiation', 'dynamic_ERA5_surface_net_thermal_radiation']:
        catmt_slice[varname] = catmt_slice[varname] / 24

    rename_columns={
        'dynamic_ERA5_surface_pressure': 'surface_pressure_mean',
        'dynamic_ERA5_2m_temperature': 'temperature_2m_mean',
        'dynamic_ERA5_2m_dewpoint_temperature': 'dewpoint_temperature_2m_mean',
        'dynamic_ERA5_10m_u_component_of_wind': 'u_component_of_wind_10m_mean',
        'dynamic_ERA5_10m_v_component_of_wind': 'v_component_of_wind_10m_mean',
        'dynamic_ERA5_surface_net_solar_radiation': 'surface_net_solar_radiation_mean',
        'dynamic_ERA5_surface_net_thermal_radiation': 'surface_net_thermal_radiation_mean',
        'dynamic_ERA5_total_precipitation': 'total_precipitation_sum',
        'dynamic_ERA5_potential_evaporation': 'potential_evaporation_sum_ERA5_LAND',
    }
    catmt_slice = catmt_slice.rename(rename_columns)
    catmt_slice = catmt_slice[sorted(catmt_slice.data_vars)]

    catmt_slice['potential_evaporation_sum_ERA5_LAND'] = catmt_slice['potential_evaporation_sum_ERA5_LAND'] * 1e3 * -1
    catmt_slice['total_precipitation_sum'] = catmt_slice['total_precipitation_sum'] * 1e3  # Convert from m to mm

    def process_catmt_idx(idx):
        catmt_slice_idx = catmt_slice.sel(idx=idx).drop_vars(['idx']).to_dataframe()
        ET0 = get_fao_pet(
            surface_pressure_mean=catmt_slice_idx['surface_pressure_mean'],
            temperature_2m_mean=catmt_slice_idx['temperature_2m_mean'],
            dewpoint_temperature_2m_mean=catmt_slice_idx['dewpoint_temperature_2m_mean'],
            u_component_of_wind_10m_mean=catmt_slice_idx['u_component_of_wind_10m_mean'],
            v_component_of_wind_10m_mean=catmt_slice_idx['v_component_of_wind_10m_mean'],
            surface_net_solar_radiation_mean=catmt_slice_idx['surface_net_solar_radiation_mean'],
            surface_net_thermal_radiation_mean=catmt_slice_idx['surface_net_thermal_radiation_mean']
        )
        catmt_slice_idx['potential_evaporation_sum_FAO_PENMAN_MONTEITH'] = ET0
        climate_indices_idx = get_climate_indices(catmt_slice_idx)
        return pd.Series(climate_indices_idx, name=idx)

    with Parallel(n_jobs=32, verbose=0) as parallel:
        climate_indices = parallel(delayed(process_catmt_idx)(idx) for idx in catmt_slice.idx.values)

    climate_indices = pd.DataFrame(climate_indices).T

    for climidx in climate_indices.index:
        catmt[f"static_ClimSumm_{climidx}"] = xr.DataArray(
            climate_indices.loc[climidx].values,
            dims=['idx'],
            coords={'idx': catmt.idx.values}
        )
    
    del climate_indices
    gc.collect()

    catmt.to_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'), mode = 'a')

In [64]:
for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
    huc, gauge_id = row['huc_02'], row['gauge_id']
    process_catmt(huc, gauge_id)

100%|██████████| 395/395 [38:42<00:00,  5.88s/it] 


In [65]:
row = camels_graph.iloc[0]
huc, gauge_id = row['huc_02'], row['gauge_id']
catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
data_vars = list(sorted(catmt.data_vars))
# print(f"Length of idx vars: {len(catmt.idx.values)}")
catmt.close()
print(f"Length of data_vars: {len(data_vars)}")
data_vars

Length of data_vars: 152


['dynamic_Daymet_dayl',
 'dynamic_Daymet_prcp',
 'dynamic_Daymet_srad',
 'dynamic_Daymet_swe',
 'dynamic_Daymet_tmax',
 'dynamic_Daymet_tmin',
 'dynamic_Daymet_vp',
 'dynamic_ERA5-Land_dewpoint_temperature_2m_max',
 'dynamic_ERA5-Land_dewpoint_temperature_2m_min',
 'dynamic_ERA5-Land_leaf_area_index_high_vegetation',
 'dynamic_ERA5-Land_leaf_area_index_low_vegetation',
 'dynamic_ERA5-Land_potential_evaporation_sum',
 'dynamic_ERA5-Land_runoff_sum',
 'dynamic_ERA5-Land_snow_cover',
 'dynamic_ERA5-Land_snow_depth',
 'dynamic_ERA5-Land_snowfall_sum',
 'dynamic_ERA5-Land_snowmelt_sum',
 'dynamic_ERA5-Land_sub_surface_runoff_sum',
 'dynamic_ERA5-Land_surface_net_solar_radiation_sum',
 'dynamic_ERA5-Land_surface_net_thermal_radiation_sum',
 'dynamic_ERA5-Land_surface_pressure',
 'dynamic_ERA5-Land_surface_runoff_sum',
 'dynamic_ERA5-Land_temperature_2m_max',
 'dynamic_ERA5-Land_temperature_2m_min',
 'dynamic_ERA5-Land_total_evaporation_sum',
 'dynamic_ERA5-Land_total_precipitation_sum',
 'dy

# Target Related Variables

In [18]:
def process_related_vars(huc, gauge_id):
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', huc, f'{gauge_id}.zarr'))
    catmt = catmt[sorted(catmt.data_vars)]

    START_DATE = pd.Timestamp('1999-01-01')
    END_DATE = pd.Timestamp('2019-12-31')
    catmt_slice = catmt.sel(time=slice(START_DATE, END_DATE)).copy()

    ERA5_var_names = [
        'dynamic_ERA5_total_precipitation',
        'dynamic_ERA5_potential_evaporation',
        'dynamic_ERA5_2m_temperature',
        'dynamic_ERA5_surface_pressure',
        'dynamic_ERA5_2m_dewpoint_temperature',
        'dynamic_ERA5_10m_u_component_of_wind',
        'dynamic_ERA5_10m_v_component_of_wind',
        'dynamic_ERA5_surface_net_solar_radiation',
        'dynamic_ERA5_surface_net_thermal_radiation'
    ]

    catmt_slice = catmt_slice[ERA5_var_names]
    for varname in ['dynamic_ERA5_surface_net_solar_radiation', 'dynamic_ERA5_surface_net_thermal_radiation']:
        catmt_slice[varname] = catmt_slice[varname] / 24

    rename_columns={
        'dynamic_ERA5_surface_pressure': 'surface_pressure_mean',
        'dynamic_ERA5_2m_temperature': 'temperature_2m_mean',
        'dynamic_ERA5_2m_dewpoint_temperature': 'dewpoint_temperature_2m_mean',
        'dynamic_ERA5_10m_u_component_of_wind': 'u_component_of_wind_10m_mean',
        'dynamic_ERA5_10m_v_component_of_wind': 'v_component_of_wind_10m_mean',
        'dynamic_ERA5_surface_net_solar_radiation': 'surface_net_solar_radiation_mean',
        'dynamic_ERA5_surface_net_thermal_radiation': 'surface_net_thermal_radiation_mean',
        'dynamic_ERA5_total_precipitation': 'total_precipitation_sum',
        'dynamic_ERA5_potential_evaporation': 'potential_evaporation_sum_ERA5_LAND',
    }
    catmt_slice = catmt_slice.rename(rename_columns)
    catmt_slice = catmt_slice[sorted(catmt_slice.data_vars)]

    catmt_slice['potential_evaporation_sum_ERA5_LAND'] = catmt_slice['potential_evaporation_sum_ERA5_LAND'] * 1e3 * -1
    catmt_slice['total_precipitation_sum'] = catmt_slice['total_precipitation_sum'] * 1e3  # Convert from m to mm

    PET = get_fao_pet(
        surface_pressure_mean=catmt_slice['surface_pressure_mean'],
        temperature_2m_mean=catmt_slice['temperature_2m_mean'],
        dewpoint_temperature_2m_mean=catmt_slice['dewpoint_temperature_2m_mean'],
        u_component_of_wind_10m_mean=catmt_slice['u_component_of_wind_10m_mean'],
        v_component_of_wind_10m_mean=catmt_slice['v_component_of_wind_10m_mean'],
        surface_net_solar_radiation_mean=catmt_slice['surface_net_solar_radiation_mean'],
        surface_net_thermal_radiation_mean=catmt_slice['surface_net_thermal_radiation_mean']
    )

    Prcp = catmt_slice['total_precipitation_sum']

    return PET.values, Prcp.values

PET_list = []
Prcp_list = []
for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)):
    huc, gauge_id = row['huc_02'], row['gauge_id']
    PET, Prcp = process_related_vars(huc, gauge_id)
    PET_list.append(PET)
    Prcp_list.append(Prcp)

PET_array = np.array(PET_list)
Prcp_array = np.array(Prcp_list)

100%|██████████| 395/395 [03:05<00:00,  2.13it/s]


In [19]:
import torch

PET_tensor = torch.tensor(PET_array, dtype=torch.float32)
Prcp_tensor = torch.tensor(Prcp_array, dtype=torch.float32)
print(PET_tensor.shape, Prcp_tensor.shape)

torch.Size([395, 7665]) torch.Size([395, 7665])


In [21]:
timestamps = torch.load('/home/sarth/rootdir/datadir/data/datasets/batched_catchments/CAMELS-US_HUCAll_lumped/timestamps.pt')
timestamps = pd.DataFrame(index=timestamps)
timestamps.index.name = 'timestamp'
timestamps

""
timestamp
1998-01-02
1998-01-03
1998-01-04
1998-01-05
1998-01-06
...
2020-01-06
2020-01-07
2020-01-08


In [22]:
start_idx = timestamps.index.get_loc(pd.Timestamp('1999-01-01'))
end_idx = timestamps.index.get_loc(pd.Timestamp('2019-12-31'))
start_idx, end_idx

(364, 8028)

In [23]:
# Create tensor same length as timestamps and fill the values from start_idx to end_idx
PET = torch.zeros((PET_tensor.shape[0], len(timestamps)), dtype=torch.float32)
PET[:, start_idx:end_idx+1] = PET_tensor
Prcp = torch.zeros((Prcp_tensor.shape[0], len(timestamps)), dtype=torch.float32)
Prcp[:, start_idx:end_idx+1] = Prcp_tensor
print(PET.shape, Prcp.shape)

torch.Size([395, 8039]) torch.Size([395, 8039])


In [25]:
SAVE_PATH = os.path.join(PATHS['datasets'], 'batched_catchments', 'CAMELS-US_HUCAll_lumped')
torch.save(PET, os.path.join(SAVE_PATH, 'PET.pt'))
torch.save(Prcp, os.path.join(SAVE_PATH, 'Prcp.pt'))

# Development

In [6]:
camels_graph = camels_graph[camels_graph['huc_02'] == '05']

In [8]:
row = camels_graph.iloc[0]
huc, gauge_id = row['huc_02'], row['gauge_id']
print(f"Processing HUC: {huc}, Gauge ID: {gauge_id}")

Processing HUC: 05, Gauge ID: 03164000


In [9]:
catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
catmt = catmt[sorted(catmt.data_vars)]
catmt

<xarray.Dataset> Size: 512MB
Dimensions:                                              (time: 9125, idx: 124,
                                                          lead: 10, lat: 14,
                                                          lon: 17)
Coordinates:
  * lead                                                 (lead) int32 40B 24 ...
  * time                                                 (time) datetime64[ns] 73kB ...
  * lat                                                  (lat) float32 56B 36...
  * lon                                                  (lon) float32 68B -8...
  * idx                                                  (idx) int32 496B 0 ....
Data variables: (12/138)
    dynamic_Daymet_dayl                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_mean                              (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_upa_std                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_50%                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_mean                              (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_std                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_uparea                                        (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03164000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[123, 122], [122, 120], [121, 120], [120, 116], [1...
    drainage_area_km2:    2963.7

In [19]:
START_DATE = pd.Timestamp('1999-01-01')
END_DATE = pd.Timestamp('2009-12-31')

catmt = catmt.sel(time=slice(START_DATE, END_DATE))
catmt

<xarray.Dataset> Size: 225MB
Dimensions:                                              (time: 4015, idx: 124,
                                                          lead: 10, lat: 14,
                                                          lon: 17)
Coordinates:
  * lead                                                 (lead) int32 40B 24 ...
  * time                                                 (time) datetime64[ns] 32kB ...
  * lat                                                  (lat) float32 56B 36...
  * lon                                                  (lon) float32 68B -8...
  * idx                                                  (idx) int32 496B 0 ....
Data variables: (12/138)
    dynamic_Daymet_dayl                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_mean                              (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_upa_std                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_50%                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_mean                              (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_std                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_uparea                                        (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03164000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[123, 122], [122, 120], [121, 120], [120, 116], [1...
    drainage_area_km2:    2963.7

In [ ]:
# list(catmt.data_vars)

In [51]:
ERA5_var_names = [
    'dynamic_ERA5_total_precipitation',
    'dynamic_ERA5_potential_evaporation',
    'dynamic_ERA5_2m_temperature',
    'dynamic_ERA5_surface_pressure',
    'dynamic_ERA5_2m_dewpoint_temperature',
    'dynamic_ERA5_10m_u_component_of_wind',
    'dynamic_ERA5_10m_v_component_of_wind',
    'dynamic_ERA5_surface_net_solar_radiation',
    'dynamic_ERA5_surface_net_thermal_radiation'
]

catmt_slice = catmt[ERA5_var_names].copy()
for varname in ['dynamic_ERA5_surface_net_solar_radiation', 'dynamic_ERA5_surface_net_thermal_radiation']:
    catmt_slice[varname] = catmt_slice[varname] / 24

rename_columns={
    'dynamic_ERA5_surface_pressure': 'surface_pressure_mean',
    'dynamic_ERA5_2m_temperature': 'temperature_2m_mean',
    'dynamic_ERA5_2m_dewpoint_temperature': 'dewpoint_temperature_2m_mean',
    'dynamic_ERA5_10m_u_component_of_wind': 'u_component_of_wind_10m_mean',
    'dynamic_ERA5_10m_v_component_of_wind': 'v_component_of_wind_10m_mean',
    'dynamic_ERA5_surface_net_solar_radiation': 'surface_net_solar_radiation_mean',
    'dynamic_ERA5_surface_net_thermal_radiation': 'surface_net_thermal_radiation_mean',
    'dynamic_ERA5_total_precipitation': 'total_precipitation_sum',
    'dynamic_ERA5_potential_evaporation': 'potential_evaporation_sum_ERA5_LAND',
}
catmt_slice = catmt_slice.rename(rename_columns)
catmt_slice = catmt_slice[sorted(catmt_slice.data_vars)]

catmt_slice['potential_evaporation_sum_ERA5_LAND'] = catmt_slice['potential_evaporation_sum_ERA5_LAND'] * 1e3 * -1
catmt_slice['total_precipitation_sum'] = catmt_slice['total_precipitation_sum'] * 1e3  # Convert from m to mm

def process_catmt_idx(idx):
    catmt_slice_idx = catmt_slice.sel(idx=idx).drop_vars(['idx']).to_dataframe()
    ET0 = get_fao_pet(
        surface_pressure_mean=catmt_slice_idx['surface_pressure_mean'],
        temperature_2m_mean=catmt_slice_idx['temperature_2m_mean'],
        dewpoint_temperature_2m_mean=catmt_slice_idx['dewpoint_temperature_2m_mean'],
        u_component_of_wind_10m_mean=catmt_slice_idx['u_component_of_wind_10m_mean'],
        v_component_of_wind_10m_mean=catmt_slice_idx['v_component_of_wind_10m_mean'],
        surface_net_solar_radiation_mean=catmt_slice_idx['surface_net_solar_radiation_mean'],
        surface_net_thermal_radiation_mean=catmt_slice_idx['surface_net_thermal_radiation_mean']
    )
    catmt_slice_idx['potential_evaporation_sum_FAO_PENMAN_MONTEITH'] = ET0
    climate_indices_idx = get_climate_indices(catmt_slice_idx)
    return pd.Series(climate_indices_idx, name=idx)

with Parallel(n_jobs=-1, verbose=10) as parallel:
    climate_indices = parallel(delayed(process_catmt_idx)(idx) for idx in tqdm.tqdm(catmt_slice.idx.values, desc="Processing indices"))

climate_indices = pd.DataFrame(climate_indices).T

for climidx in climate_indices.index:
    catmt[f"static_ClimSumm_{climidx}"] = xr.DataArray(
        climate_indices.loc[climidx].values,
        dims=['idx'],
        coords={'idx': catmt.idx.values}
    )

catmt



Processing indices: 100%|██████████| 124/124 [00:01<00:00, 90.08it/s]
[Parallel(n_jobs=-1)]: Done  10 out of 124 | elapsed:    4.1s remaining:   46.5s
[Parallel(n_jobs=-1)]: Done  23 out of 124 | elapsed:    4.2s remaining:   18.6s
[Parallel(n_jobs=-1)]: Done  36 out of 124 | elapsed:    4.4s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done  49 out of 124 | elapsed:    4.5s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  62 out of 124 | elapsed:    4.6s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done  75 out of 124 | elapsed:    4.6s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  88 out of 124 | elapsed:    4.7s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done 101 out of 124 | elapsed:    4.8s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done 114 out of 124 | elapsed:    4.9s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 124 out of 124 | elapsed:    4.9s finished


<xarray.Dataset> Size: 225MB
Dimensions:                                              (time: 4015, idx: 124,
                                                          lead: 10, lat: 14,
                                                          lon: 17)
Coordinates:
  * lead                                                 (lead) int32 40B 24 ...
  * time                                                 (time) datetime64[ns] 32kB ...
  * lat                                                  (lat) float32 56B 36...
  * lon                                                  (lon) float32 68B -8...
  * idx                                                  (idx) int32 496B 0 ....
Data variables: (12/152)
    dynamic_Daymet_dayl                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time, idx) float32 2MB dask.array<chunksize=(1917, 62), meta=np.ndarray>
    ...                                                   ...
    static_ClimSumm_moisture_index_FAO_PM                (idx) float64 992B 0...
    static_ClimSumm_seasonality_FAO_PM                   (idx) float64 992B 0...
    static_ClimSumm_high_prec_freq                       (idx) float64 992B 0...
    static_ClimSumm_high_prec_dur                        (idx) float64 992B 1...
    static_ClimSumm_low_prec_freq                        (idx) float64 992B 0...
    static_ClimSumm_low_prec_dur                         (idx) float64 992B 3...
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03164000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[123, 122], [122, 120], [121, 120], [120, 116], [1...
    drainage_area_km2:    2963.7

In [52]:
138 + 14

152

In [23]:
ERA5_LAND_var_names = [
    'dynamic_ERA5-Land_total_precipitation_sum',
    'dynamic_ERA5-Land_potential_evaporation_sum',
    'dynamic_ERA5-Land_temperature_2m_max',
    'dynamic_ERA5-Land_temperature_2m_min',
    'dynamic_ERA5-Land_surface_pressure',
    'dynamic_ERA5-Land_dewpoint_temperature_2m_max',
    'dynamic_ERA5-Land_dewpoint_temperature_2m_min',
    'dynamic_ERA5-Land_u_component_of_wind_10m',
    'dynamic_ERA5-Land_v_component_of_wind_10m',
    'dynamic_ERA5-Land_surface_net_solar_radiation_sum',
    'dynamic_ERA5-Land_surface_net_thermal_radiation_sum'
]

In [58]:
len(list(catmt.data_vars)) == 152

True